Topological insulator is characterized by $Z_2$ invariant defined as $Z_2$ invariant
$$
Z_2=\sum_n^{o c c .} \frac{1}{2 \pi}\left(\oint_{\text {Half } \mathrm{BZ}} \mathbf{A}_n \cdot d \mathbf{k}-\int_{\text {Half } \mathrm{BZ}} d k_x d k_y F_n\right)=0 \operatorname{or} 1 (\bmod 2)
$$
$Z_2=0$ corresponds to a trivial state and
$Z_2=1$ corresponds to a topological insulator state.

In [51]:
import numpy as np

In [52]:
def QWZ_hamiltonian(kx, ky, t, m):
    """
    QWZ model Hamiltonian (2 x 2).
    kx, ky: momentum components
    t: hopping amplitude
    m: mass term
    """
    sigma_x = np.array([[0, 1], 
                        [1, 0]], dtype=complex)
    sigma_y = np.array([[0, -1j], 
                        [1j,  0]], dtype=complex)
    sigma_z = np.array([[1,  0], 
                        [0, -1]], dtype=complex)
    
    # 对应系数
    Hy = t * np.sin(kx)           # 系数乘 sigma_x
    Hz = t * np.sin(ky)           # 系数乘 sigma_y
    Hx = m + t * np.cos(kx) + t * np.cos(ky)  # 系数乘 sigma_z

    # 构造 2x2 哈密顿量
    H = Hx * sigma_x + Hy * sigma_y + Hz * sigma_z
    return H

In [91]:
def QWZ2layer_tri_hamiltonian(kx, ky):
    """
    Example placeholder for a time-reversal-invariant (TRI) 2D Hamiltonian.
    Must return a (2n x 2n) Hermitian matrix, e.g. a spinful 2-band model => 4x4.
    """
    # --- Replace with your actual model (e.g. Kane-Mele, BHZ, etc.) ---
    
    # For demonstration, return a 2-layer QWZ model
    H1 = QWZ_hamiltonian(kx, ky, t=1, m=1)
    H2 = QWZ_hamiltonian(kx, ky, t=1, m=-1)
    H = np.zeros((4, 4), dtype=complex)
    H[:2, :2] = H1
    H[2:, 2:] = H2
    H[:2,2:] = np.eye(2) * 0.1
    H[2:,:2] = np.eye(2) * 0.1
    # ----------------------------------------------------------------
    
    return H

def get_occupied_subspace(H, n_occ):
    """
    Diagonalize H and return the subspace (2n x n_occ) of the lowest n_occ eigenstates.
    """
    vals, vecs = np.linalg.eig(H)
    idx = np.argsort(vals.real)
    return vecs[:, idx[:n_occ]]  # columns = occupied eigenvectors

def overlap_det(U1, U2):
    """
    Compute the determinant of the overlap matrix between two subspaces U1, U2.
    U1, U2: (2n x n_occ) each.
    Overlap matrix O = U1^dagger * U2 => (n_occ x n_occ).
    """
    O = np.dot(np.conjugate(U1.T), U2)
    return np.linalg.det(O)

def z2_invariant_fukui(hamiltonian, Nk=80, n_occ=4):
    """
    Compute the Z2 invariant using a Fukui-Hatsugai-like approach.
    We take kx in [0, 2pi], ky in [0, pi].
    
    Nk: number of points in the kx direction.
        We will use Nk//2 points in the ky direction.
    n_occ: number of occupied states (e.g., 2 for one Kramers pair, 4 for two pairs, etc.)
    
    Returns 0 or 1 (the Z2 index).
    """
    dx = 2.0 * np.pi / Nk        # step in kx
    dy = np.pi / (Nk // 2)       # step in ky
    
    total_flux = 0.0
    
    for i in range(Nk):
        for j in range(Nk // 2):
            kx = i * dx
            ky = j * dy
            
            # Occupied subspaces at corners of the plaquette
            U1 = get_occupied_subspace(hamiltonian(kx,        ky),        n_occ)
            U2 = get_occupied_subspace(hamiltonian(kx + dx,   ky),        n_occ)
            U3 = get_occupied_subspace(hamiltonian(kx + dx,   ky + dy),   n_occ)
            U4 = get_occupied_subspace(hamiltonian(kx,        ky + dy),   n_occ)
            
            # Determinants of overlap matrices
            U12 = overlap_det(U1, U2)
            U23 = overlap_det(U2, U3)
            U34 = overlap_det(U3, U4)
            U41 = overlap_det(U4, U1)
            
            # Discrete Berry curvature for this plaquette
            prod = U12 * U23 * U34 * U41
            flux = -np.angle(prod) + np.angle(U12) + np.angle(U23) + np.angle(U34) + np.angle(U41)
            
            total_flux += flux
            
    
    # Normalize by 2*pi to get an integer => take mod 2 for the Z2 index
    z2_raw = total_flux / (2.0 * np.pi)
    z2_value = int(round(z2_raw)) % 2
    
    return z2_value, z2_raw


In [86]:
if __name__ == "__main__":
    # Example usage:
    # Suppose a 4x4 Hamiltonian with 2 occupied states (n_occ=2).
    Nk = 80  # kx steps; ky steps = Nk//2
    z2, z2_raw = z2_invariant_fukui(QWZ2layer_tri_hamiltonian, Nk=Nk, n_occ=2)
    print("Z2 invariant =", z2)
    print("Raw flux =", z2_raw)

Z2 invariant = 1
Raw flux = -1.0


In [100]:
def QWZ4layer_tri_hamiltonian(kx, ky):
    """
    Example placeholder for a time-reversal-invariant (TRI) 2D Hamiltonian.
    Must return a (2n x 2n) Hermitian matrix, here let me build a 8x8 model, which corresponds to 4 layers of QWZ model. \
    """
    
    H1 = QWZ_hamiltonian(kx, ky, t=1, m=1)
    H2 = QWZ_hamiltonian(kx, ky, t=1, m=1)
    H3 = QWZ_hamiltonian(kx, ky, t=1, m=-1)
    H4 = QWZ_hamiltonian(kx, ky, t=1, m=-1)
    H = np.zeros((8, 8), dtype=complex)
    H[:2, :2] = H1
    H[2:4, 2:4] = H2
    H[4:6, 4:6] = H3
    H[6:, 6:] = H4
    
    # only connect opopsite m layers
    delta = 0.6
    H[:2, 4:6] = np.eye(2) * delta
    H[2:4, 6:] = np.eye(2) * delta
    H[:2, 6:8] = np.eye(2) * delta
    H[6:8, :2] = np.eye(2) * delta
    H[2:4, 4:6] = np.eye(2) * delta
    H[4:6, 2:4] = np.eye(2) * delta
    H[2:4, 6:8] = np.eye(2) * delta
    H[6:8, 2:4] = np.eye(2) * delta
    
    # ----------------------------------------------------------------
    
    return H

In [101]:
if __name__ == "__main__":
    Nk = 80  # kx steps; ky steps = Nk//2
    z2, z2_raw = z2_invariant_fukui(QWZ4layer_tri_hamiltonian, Nk=Nk, n_occ=4)
    print("Z2 invariant =", z2)
    print("Raw flux =", z2_raw)

Z2 invariant = 1
Raw flux = -0.9999999999999927
